# Multisample branch: Master Workflow for single-normal multi-tumor

## Availalbe multilane workflows ( Array[Array[File]] )  in ../src/wdl/
* trimming (Optional)
    * Inputs are multi-lane fasta
    * Alignment/Workflow/Runtrim_sequences.wdl
    * Outputs are multi-lane
* alignment
    * Input option (select_first) inputs are multi-lane fasta
    * Alignment/Workflow/Runalignment.wdl
    * Outputs are multi-lane fasta
* merge
    * inputs are multi-lane fasta
    * Alignment/Workflow/RunMergeAligned.wdl
    * Outputs: Bam, Bai

## Available (Bam-Bai) merged functions
* dedup (Optional)
    * Inputs are Bam, Bai
    * Alignment/Tasks/dedup.wdl
    * Outputs: - Bam, Bai
* deliver alignment
    * Input option (select_first) Bam, Bai
    * DeliveryOfAlignment/Tasks/deliver_alignment.wdl
    * Output - script delivers to:              DeliveryFolder_Alignment
* realignment
    * Inputs: DeliverAlignOutputBams, DeliverAlignOutputBais
    * HaplotyperVC/Tasks/realignment.wdl
    * Outputs: OutputBams, OutPutBais
* Bqsr (Optional)
    * Inputs OutputBams, OutPutBais
    * HaplotyperVC/Tasks/bqsr.wdl
    * Outputs RecalTable, RecalibratedBam, RecalibratedBai
* haplotyper
    * Inputs (select_first) and RecalTable
    * HaplotyperVC/Tasks/haplotyper.wdl
    * Outputs: OutputVcf, OutputVcfIdx
* Vqsr (Optional)
    * Inputs: OutputVcf, OutputVcfIdx
    * HaplotyperVC/Tasks/vqsr.wdl
    * Outputs: OutputVcf, OutputVcfIdx
* deliver HaplotyperVC
    * Inputs: (select_first)
    * DeliveryOfHaplotyperVC/Tasks/deliver_HaplotyperVC.wdl
    * Outputs: script delivery to DeliveryFolder_HaplotyperVC
****

# Multi-sample, single normal workflow strategy
## Reuse the workflow of multilane workflows through merge
* call once with normal
* scatter with multisamples
## Use merged workflow *SomaticVCStandaloneWorkflow.wdl*
* scatter each tumor with normal
* delivery task: what is the form of final output ?
****

```bash
###############################################################################################
##		SomaticMasterMultiWorkflow.wdl        ##
###############################################################################################


import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunTrimSequences_MultiSample.wdl" as TRIM_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunAlignment_MultiSample.wdl" as ALIGN_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunMergeAligned_MultiSample.wdl" as MERGE_MULTI_SAMPLE

workflow SomaticVCMultiSampleMasterWF {
        
   Array[Array[Array[File]]] InputReads 
	
   call TRIM_MULTI_SAMPLE_AND_LANE.RunMultiSample_Trimming as TrimMultiSampleMultiLane {
      input:
         InputReads = InputReads	
   }

   call ALIGN_MULTI_SAMPLE_AND_LANE.RunMultiSample_Alignment as AlignMultiSampleMultiLane {
      input:
         InputReads = TrimMultiSampleMultiLane.Outputs
   }

   call MERGE_MULTI_SAMPLE.RunMultiSampleMerge as MergeMultiSample {
      input:
         AlignedMultiSampleOutputBams = AlignMultiSampleMultiLane.OutputBams,
         AlignedMultiSampleOutputBais = AlignMultiSampleMultiLane.OutputBais
   }
}
```
****
## dedup & deliver -- then scatter pairs on *SomaticVCStandaloneWorkflow.wdl*
****

```bash
###############################################################################################
##		SomaticMultiSample.wdl       ##
###############################################################################################


# import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunTrimSequences_MultiSample.wdl" as TRIM_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/Alignment/Workflow/Runtrim_sequences.wdl" as TRIM_MULTILANE
#import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunAlignment_MultiSample.wdl" as ALIGN_MULTI_SAMPLE_AND_LANE
import "MayomicsVC/src/wdl/Alignment/Workflow/Runalignment.wdl" as ALIGN_MULTILANE
# import "MayomicsVC/src/wdl/SomaticVC/Workflow/RunMergeAligned_MultiSample.wdl" as MERGE_MULTI_SAMPLE
import "MayomicsVC/src/wdl/Alignment/Workflow/RunMergeAligned.wdl" as MERGE_MULTI

workflow SomaticVCWorkflow {

   Array[Array[File]] NormalInputReads
   Array[Array[Array[File]]] TumorInputReads
   Array[String] PairSamplesName

   full_idx = length(TumorInputReads)
   Array[Int] Index = range(full_idx)

   scatter (idx in Index) {
   
      call TRIM_MULTILANE.RunTrimSequencesTask as TRIM_MULTISAMPLE {
         input:
            PairedEnd = PairedEnd,
            InputReads = InputReads[idx],
            SampleName = SampleName[idx]
      }
      
      # select_first(flatten(...
      call ALIGN_MULTILANE.RunAlignmentTask as ALIGN_MULTISAMPLE {
         input:
            PairedEnd = PairedEnd,
            InputReads = InputReads[idx],
            SampleName = SampleName[idx],
            PlatformUnit = PlatformUnit
      }
      # flatten
      call MERGE_MULTI.RunMergeAlignedBamsTask as MERGE_MULTI {
         input:
            SampleName = SampleName[idx],
            AlignedOutputBams = AlignedMultiSampleOutputBams[idx],
            AlignedOutputBais = AlignedMultiSampleOutputBais[idx]
      }
      # flatten
      # if dedup - dedup
      # select_first(flatten(...
      # deliver alaignment
      #      
      #      begin with bam-bai stuff -------------------------------------
   }

#    call TRIM_MULTI_SAMPLE_AND_LANE.RunMultiSample_Trimming as TrimMultiSampleMultiLane {
#       input:
#          InputReads = InputReads	
#    }

#    call ALIGN_MULTI_SAMPLE_AND_LANE.RunMultiSample_Alignment as AlignMultiSampleMultiLane {
#       input:
#          InputReads = TrimMultiSampleMultiLane.Outputs
#    }

#    call MERGE_MULTI_SAMPLE.RunMultiSampleMerge as MergeMultiSample {
#       input:
#          AlignedMultiSampleOutputBams = AlignMultiSampleMultiLane.OutputBams,
#          AlignedMultiSampleOutputBais = AlignMultiSampleMultiLane.OutputBais
#    }
}
```